In [1]:
import numpy as np
import pandas as pd
import os
from datetime import date
import sqlalchemy
import pymysql
import re
import matplotlib.pyplot as plt

import warnings    # to avoid warning during executions
warnings.filterwarnings("ignore")

In [2]:
# load all the file names into a list (files in the pbr data folder)
os.getcwd()
_dir_path_day = '../../../dataExport/cumulativeDATA/Day'
raw_data_day = os.listdir(_dir_path_day)
raw_data_day

['_10_FPC23_RhoSa.csv',
 '_11_FPC24_SkeMa_1.csv',
 '_12_FPC24_SkeMa_2.csv',
 '_16_FPC23_NaCho.csv',
 '_1_FPC13_ChCal_1.csv',
 '_2_FPC13_ChCal_2.csv',
 '_3_FPC13_ThalaPs.csv',
 '_4_FPC14_ChaMu.csv',
 '_5_FPC14_RhoSa.csv',
 '_6_FPC14_ThalaPs.csv',
 '_7_FPC21_ChaMu.csv',
 '_8_FPC21_TeChu.csv',
 '_9_FPC22_DiaLut.csv']

In [3]:
filename_day = raw_data_day[6]
filename_day

'_3_FPC13_ThalaPs.csv'

In [4]:
# join file name with path - file name can be specified as list item based on required file name
file_path_day=  os.path.join(_dir_path_day, filename_day)

In [5]:
sum_Day_raw = pd.read_csv(file_path_day, sep=',')

In [6]:
# database connection with mariaDB for read the read and write data from and to dataframe <----> mariaDB tables
database_username = 'root'
database_password = 'password'
database_ip       = '127.0.0.1:3306'
database_name     = 'data_dashboard'
database_connection = sqlalchemy.create_engine('mariadb+mariadbconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [7]:
trial_no = re.findall(r'\d+', filename_day)
trial_no = int(trial_no[0])
trial_no

3

In [8]:
tbl_lookup = pd.read_sql("SELECT * FROM tbl_pbr_lookup WHERE instance = ?", database_connection, params=[trial_no])

In [9]:
tbl_lookup

,instance,unit,species,start_date,end_date,remark,raw_file_name
0,3,6,11,1625227200,1628157595,None,_3_FPC13_ThalaPs.csv


In [10]:
species_id = int(tbl_lookup['species'])
trial_no = int(tbl_lookup['instance'])
unit_id = int(tbl_lookup['unit'])
start_date = int(tbl_lookup['start_date'])
end_date = int(tbl_lookup['end_date'])
parameter = 12
category = 6

In [11]:
print(start_date)
print(end_date)

1625227200
1628157595


In [12]:
tbl_log_calc = pd.read_sql("SELECT * FROM tbl_log_data WHERE unit = ? AND parameter = ? AND category = ? AND time_epoch BETWEEN ? AND ?", database_connection, params=[unit_id, parameter, category, start_date, end_date])

In [13]:
tbl_log_calc.head(20)

,date_time,time_epoch,category,unit,parameter,message,new_value,old_value
0,2021-07-03 05:00:00,1625281200,6,6,12,Daily PAR(mol/m),empty,18.9
1,2021-07-03 05:00:00,1625281200,6,6,12,Daily productivity (g/m),empty,9.5
2,2021-07-03 05:00:00,1625281200,6,6,12,Daily productivity (g),empty,1.89
3,2021-07-03 05:00:00,1625281200,6,6,12,Daily efficiency (%),empty,2.3
4,2021-07-03 05:00:00,1625281200,6,6,12,Daily CO2 consumption(g/m),empty,27
5,2021-07-03 05:00:00,1625281200,6,6,12,Daily CO2 consumption(g),empty,5.37
6,2021-07-03 05:00:00,1625281200,6,6,12,Daily CO2 conversion(g/g),empty,2.8
7,2021-07-03 05:00:00,1625281200,6,6,12,Daily Feed(L),empty,4.571
8,2021-07-03 05:00:00,1625281200,6,6,12,Daily Harvest(L),empty,0.000
9,2021-07-03 05:00:00,1625281200,6,6,12,Daily Harvested algae(g),empty,0.000


In [14]:
sum_daily = tbl_log_calc.pivot_table(index=(['date_time', 'time_epoch', 'category', 'unit', 'parameter']),columns='message',values='old_value',fill_value=0)

In [15]:
sum_daily

,,,,message,CD (g/l),CO2 to algae conversion,Daily CO2 consumption(g),Daily CO2 consumption(g/m),Daily CO2 conversion(g/g),Daily Feed(L),Daily Harvest(L),Daily Harvested algae(g),Daily PAR(mol/m),Daily efficiency (%),Daily productivity (g),Daily productivity (g/m),PAR to algae conversion,connected reactor surface (m),reactor biomassa (kg),reactor volume (l)
date_time,time_epoch,category,unit,parameter,,,,,,,,,,,,,,,,
2021-07-03 05:00:00,1625281200,6,6,12,0.30,0.6945,5.37,27,2.8,4.571,0.000,0.000,18.9,2.3,1.89,9.5,0.07961,0.2,0.0018,6.00
2021-07-04 05:00:00,1625367600,6,6,12,0.82,0.6945,10.42,52,2.8,1.155,0.885,0.345,34.0,2.4,3.68,18.4,0.07961,0.2,0.0052,6.27
2021-07-05 05:00:00,1625454000,6,6,12,0.94,0.6945,9.12,46,2.9,2.572,2.468,2.358,34.0,2.1,3.18,15.9,0.07961,0.2,0.0060,6.37
2021-07-06 05:00:00,1625540400,6,6,12,0.99,0.6945,9.24,46,2.8,2.885,2.874,2.940,34.0,2.2,3.25,16.2,0.07961,0.2,0.0063,6.39
2021-07-07 05:00:00,1625626800,6,6,12,1.00,0.6945,9.24,46,2.8,3.005,3.090,3.216,34.0,2.2,3.25,16.3,0.07961,0.2,0.0063,6.30
2021-07-08 05:00:00,1625713200,6,6,12,1.08,0.6945,10.37,52,2.8,2.913,3.016,3.295,34.0,2.4,3.66,18.3,0.07961,0.2,0.0067,6.20
2021-07-09 05:00:00,1625799600,6,6,12,1.15,0.6945,10.74,54,2.8,2.873,2.896,3.420,34.0,2.5,3.81,19.0,0.07961,0.2,0.0071,6.17
2021-07-10 05:00:00,1625886000,6,6,12,1.19,0.6945,10.65,53,2.8,2.850,2.848,3.529,34.0,2.5,3.78,18.9,0.07961,0.2,0.0073,6.18
2021-07-11 05:00:00,1625972400,6,6,12,1.22,0.6945,10.70,54,2.8,2.864,2.954,3.746,34.0,2.5,3.82,19.1,0.07961,0.2,0.0074,6.09


In [16]:
df2 = sum_daily.reset_index()

In [17]:
df2.head(5)

message,date_time,time_epoch,category,unit,parameter,CD (g/l),CO2 to algae conversion,Daily CO2 consumption(g),Daily CO2 consumption(g/m),Daily CO2 conversion(g/g),...,Daily Harvest(L),Daily Harvested algae(g),Daily PAR(mol/m),Daily efficiency (%),Daily productivity (g),Daily productivity (g/m),PAR to algae conversion,connected reactor surface (m),reactor biomassa (kg),reactor volume (l)
0,2021-07-03 05:00:00,1625281200,6,6,12,0.30,0.6945,5.37,27,2.8,...,0.000,0.000,18.9,2.3,1.89,9.5,0.07961,0.2,0.0018,6.00
1,2021-07-04 05:00:00,1625367600,6,6,12,0.82,0.6945,10.42,52,2.8,...,0.885,0.345,34.0,2.4,3.68,18.4,0.07961,0.2,0.0052,6.27
2,2021-07-05 05:00:00,1625454000,6,6,12,0.94,0.6945,9.12,46,2.9,...,2.468,2.358,34.0,2.1,3.18,15.9,0.07961,0.2,0.0060,6.37
3,2021-07-06 05:00:00,1625540400,6,6,12,0.99,0.6945,9.24,46,2.8,...,2.874,2.940,34.0,2.2,3.25,16.2,0.07961,0.2,0.0063,6.39
4,2021-07-07 05:00:00,1625626800,6,6,12,1.00,0.6945,9.24,46,2.8,...,3.090,3.216,34.0,2.2,3.25,16.3,0.07961,0.2,0.0063,6.30


In [18]:
df2.columns


Index(['date_time', 'time_epoch', 'category', 'unit', 'parameter', 'CD (g/l)',
       'CO2 to algae conversion', 'Daily CO2 consumption(g)',
       'Daily CO2 consumption(g/m)', 'Daily CO2 conversion(g/g)',
       'Daily Feed(L)', 'Daily Harvest(L)', 'Daily Harvested algae(g)',
       'Daily PAR(mol/m)', 'Daily efficiency (%)', 'Daily productivity (g)',
       'Daily productivity (g/m)', 'PAR to algae conversion',
       'connected reactor surface (m)', 'reactor biomassa (kg)',
       'reactor volume (l)'],
      dtype='object', name='message')

In [19]:
'''
time_stamp, date_time, unit, trial_no, species

CD (g/l)                    : D_CD_gpl
CO2 to algae conversion     : Co2_Conv
Daily CO2 consumption(g)    : D_CO2_g
Daily CO2 consumption(g/m)  : D_CO2_gpm
Daily CO2 conversion(g/g)   : D_Co2_Conv_gpg
Daily Feed(L)               : D_Feed_l
Daily Harvest(L)            : D_Harvest_l
Daily Harvested algae(g)    : D_Harvest_Algae_g
Daily PAR(mol/m)            : D_PAR_molpm
Daily efficiency (%)        : D_eff_percent
Daily productivity (g)      : D_Productivity_g
Daily productivity (g/m)    : D_Productivity_gpm
PAR to algae conversion     : PAR_2Algae_Conv
reactor surface (m)         : R_Surface_m
reactor biomass (kg)        : R_Biomass_kg
reactor volume (l)          : R_Volume_l

'''

'\ntime_stamp, date_time, unit, trial_no, species\n\nCD (g/l)                    : D_CD_gpl\nCO2 to algae conversion     : Co2_Conv\nDaily CO2 consumption(g)    : D_CO2_g\nDaily CO2 consumption(g/m)  : D_CO2_gpm\nDaily CO2 conversion(g/g)   : D_Co2_Conv_gpg\nDaily Feed(L)               : D_Feed_l\nDaily Harvest(L)            : D_Harvest_l\nDaily Harvested algae(g)    : D_Harvest_Algae_g\nDaily PAR(mol/m)            : D_PAR_molpm\nDaily efficiency (%)        : D_eff_percent\nDaily productivity (g)      : D_Productivity_g\nDaily productivity (g/m)    : D_Productivity_gpm\nPAR to algae conversion     : PAR_2Algae_Conv\nreactor surface (m)         : R_Surface_m\nreactor biomass (kg)        : R_Biomass_kg\nreactor volume (l)          : R_Volume_l\n\n'

In [20]:
# swap 'date_time', 'time_epoch' columns

'''
Method 1 
Say your current order of column is [b,c,d,a] and you want to order it into [a,b,c,d], you could do it this way:
> new_df = old_df[['a', 'b', 'c', 'd']]
here we need to specify all column names. best method if we have only few column names
'''

'''
Method 2
Two column Swapping

> cols = list(df.columns)
> a, b = cols.index('LastName'), cols.index('MiddleName')
> cols[b], cols[a] = cols[a], cols[b]
> df = df[cols]
'''

'''
Method 3
Reorder column Swapping (2 swaps)

> cols = list(df.columns)
> a, b, c, d = cols.index('LastName'), cols.index('MiddleName'), cols.index('Contact'), cols.index('EmployeeID')
> cols[a], cols[b], cols[c], cols[d] = cols[b], cols[a], cols[d], cols[c]
> df = df[cols]
'''

'''
Method 4

Swapping Multiple
Now it comes down to how you can play with list slices -

cols = list(df.columns)
cols = cols[1::2] + cols[::2]
df = df[cols]
'''


cols = list(df2.columns)
a, b = cols.index('date_time'), cols.index('time_epoch')
cols[b], cols[a] = cols[a], cols[b]
cum_per_day_messyDF = df2[cols]

In [21]:
cum_per_day_messyDF.head(5)

message,time_epoch,date_time,category,unit,parameter,CD (g/l),CO2 to algae conversion,Daily CO2 consumption(g),Daily CO2 consumption(g/m),Daily CO2 conversion(g/g),...,Daily Harvest(L),Daily Harvested algae(g),Daily PAR(mol/m),Daily efficiency (%),Daily productivity (g),Daily productivity (g/m),PAR to algae conversion,connected reactor surface (m),reactor biomassa (kg),reactor volume (l)
0,1625281200,2021-07-03 05:00:00,6,6,12,0.30,0.6945,5.37,27,2.8,...,0.000,0.000,18.9,2.3,1.89,9.5,0.07961,0.2,0.0018,6.00
1,1625367600,2021-07-04 05:00:00,6,6,12,0.82,0.6945,10.42,52,2.8,...,0.885,0.345,34.0,2.4,3.68,18.4,0.07961,0.2,0.0052,6.27
2,1625454000,2021-07-05 05:00:00,6,6,12,0.94,0.6945,9.12,46,2.9,...,2.468,2.358,34.0,2.1,3.18,15.9,0.07961,0.2,0.0060,6.37
3,1625540400,2021-07-06 05:00:00,6,6,12,0.99,0.6945,9.24,46,2.8,...,2.874,2.940,34.0,2.2,3.25,16.2,0.07961,0.2,0.0063,6.39
4,1625626800,2021-07-07 05:00:00,6,6,12,1.00,0.6945,9.24,46,2.8,...,3.090,3.216,34.0,2.2,3.25,16.3,0.07961,0.2,0.0063,6.30


In [28]:
# database connection with mariaDB using SQL Alchemy
def dbConn_sqlAlc():
    database_username = 'root'
    database_password = 'password'
    database_ip       = '127.0.0.1:3306'
    database_name     = 'data_dashboard'
    database_connection = sqlalchemy.create_engine('mariadb+mariadbconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))
    return database_connection

# # database connection with mariaDB using Maria DB package
def dBCon_Maria():
    connection = pymysql.connect(host='localhost',
                            user='root',
                            password='password',
                            db='data_dashboard')
    return connection


# connection = dBCon_Maria()
# cursor=connection.cursor()                                            
# database_connection = dbConn_sqlAlc()

In [29]:
def set_varables(trial_no):
    database_connection = dbConn_sqlAlc()
    tbl_lookup = pd.read_sql("SELECT * FROM tbl_pbr_lookup WHERE instance = ?", database_connection, params=[trial_no])
    species_id = int(tbl_lookup['species'])
    trial_no = int(tbl_lookup['instance'])
    unit_id = int(tbl_lookup['unit'])
    start_date = int(tbl_lookup['start_date'])
    end_date = int(tbl_lookup['end_date'])
    parameter = 12
    category = 6
    return species_id, trial_no, unit_id, start_date, end_date, parameter, category

In [30]:
# Preprocessing the data setting/ adding values and data types
def pbr_preProcess(trial_no):
    species_id, trial_no, unit_id, start_date, end_date, parameter, category = set_varables(trial_no)
    tbl_log_calc = pd.read_sql("SELECT * FROM tbl_log_data WHERE unit = ? AND parameter = ? AND category = ? AND time_epoch BETWEEN ? AND ?", database_connection, params=[unit_id, parameter, category, start_date, end_date])
    sum_daily = tbl_log_calc.pivot_table(index=(['date_time', 'time_epoch', 'category', 'unit', 'parameter']),columns='message',values='old_value',fill_value=0)
    pbr_raw_tmp1 = sum_daily.reset_index()
    pbr_raw_tmp1= pbr_raw_tmp1.drop(columns=['category', 'parameter'])
    pbr_raw_tmp1.insert(loc=3, column='trial_no', value=trial_no)
    pbr_raw_tmp1.insert(loc=4, column='species', value=species_id)
    cols = list(pbr_raw_tmp1.columns)
    a, b = cols.index('date_time'), cols.index('time_epoch')
    cols[b], cols[a] = cols[a], cols[b]
    pbr_raw_tmp1 = pbr_raw_tmp1[cols]      
    return(
    pbr_raw_tmp1
    .rename(columns = {'time_epoch':'time_stamp', 'CD (g/l)':'D_CD_gpl', 'CO2 to algae conversion':'Co2_Conv',
                        'Daily CO2 consumption(g)':'D_CO2_g', 'Daily CO2 consumption(g/m)':'D_CO2_gpm',
                        'Daily CO2 conversion(g/g)':'D_Co2_Conv_gpg', 'Daily Feed(L)':'D_Feed_l', 'Daily Harvest(L)':'D_Harvest_l',
                        'Daily Harvested algae(g)':'D_Harvest_Algae_g', 'Daily PAR(mol/m)':'D_PAR_molpm', 'Daily efficiency (%)':'D_eff_percent',
                        'Daily productivity (g)':'D_Productivity_g', 'Daily productivity (g/m)':'D_Productivity_gpm',
                        'PAR to algae conversion':'PAR_2Algae_Conv', 'connected reactor surface (m)':'R_Surface_m',
                        'reactor biomassa (kg)':'R_Biomass_kg', 'reactor volume (l)':'R_Volume_l'})
    .assign(unit = unit_id)
    .replace('Empty', pd.np.nan)
    .replace('inf', pd.np.nan)
    .replace([np.inf, -np.inf], pd.np.nan)
    .astype({'time_stamp':'int64', 'date_time':'datetime64[ns]','unit': 'int16', 'trial_no': 'int16', 'species':'int16',
            'D_CD_gpl':'float32', 'Co2_Conv':'float32', 'D_CO2_g':'float32', 'D_CO2_gpm':'float32', 'D_Co2_Conv_gpg':'float32',
            'D_Feed_l':'float32', 'D_Harvest_l':'float32', 'D_Harvest_Algae_g':'float32', 'D_PAR_molpm':'float32', 'D_eff_percent':'float32',
            'D_Productivity_g':'float32', 'D_Productivity_gpm':'float32', 'PAR_2Algae_Conv':'float32', 'R_Surface_m':'float32',
            'R_Biomass_kg':'float32', 'R_Volume_l':'float32'})
    )
tmp_CumDay_df = pbr_preProcess(trial_no)

In [31]:
tmp_CumDay_df.head()

message,time_stamp,date_time,unit,trial_no,species,D_CD_gpl,Co2_Conv,D_CO2_g,D_CO2_gpm,D_Co2_Conv_gpg,...,D_Harvest_l,D_Harvest_Algae_g,D_PAR_molpm,D_eff_percent,D_Productivity_g,D_Productivity_gpm,PAR_2Algae_Conv,R_Surface_m,R_Biomass_kg,R_Volume_l
0,1625281200,2021-07-03 05:00:00,6,3,11,0.30,0.6945,5.37,27.0,2.8,...,0.000,0.000,18.9,2.3,1.89,9.500000,0.07961,0.2,0.0018,6.00
1,1625367600,2021-07-04 05:00:00,6,3,11,0.82,0.6945,10.42,52.0,2.8,...,0.885,0.345,34.0,2.4,3.68,18.400000,0.07961,0.2,0.0052,6.27
2,1625454000,2021-07-05 05:00:00,6,3,11,0.94,0.6945,9.12,46.0,2.9,...,2.468,2.358,34.0,2.1,3.18,15.900000,0.07961,0.2,0.0060,6.37
3,1625540400,2021-07-06 05:00:00,6,3,11,0.99,0.6945,9.24,46.0,2.8,...,2.874,2.940,34.0,2.2,3.25,16.200001,0.07961,0.2,0.0063,6.39
4,1625626800,2021-07-07 05:00:00,6,3,11,1.00,0.6945,9.24,46.0,2.8,...,3.090,3.216,34.0,2.2,3.25,16.299999,0.07961,0.2,0.0063,6.30


In [32]:
tmp_CumDay_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   time_stamp          34 non-null     int64         
 1   date_time           34 non-null     datetime64[ns]
 2   unit                34 non-null     int16         
 3   trial_no            34 non-null     int16         
 4   species             34 non-null     int16         
 5   D_CD_gpl            34 non-null     float32       
 6   Co2_Conv            34 non-null     float32       
 7   D_CO2_g             34 non-null     float32       
 8   D_CO2_gpm           34 non-null     float32       
 9   D_Co2_Conv_gpg      34 non-null     float32       
 10  D_Feed_l            34 non-null     float32       
 11  D_Harvest_l         34 non-null     float32       
 12  D_Harvest_Algae_g   34 non-null     float32       
 13  D_PAR_molpm         34 non-null     float32       
 

In [33]:
# Missing Value imputation using interpolation - spline
def missingValue_imputation(sum_hour_cleaned):
    sum_hour_cleaned_indexed = sum_hour_cleaned.set_index('date_time')
    null_cols = sum_hour_cleaned_indexed.columns[sum_hour_cleaned_indexed.isnull().any()].tolist()
    if null_cols:
        for pbr_col in null_cols:
            sum_hour_cleaned_indexed[pbr_col]=sum_hour_cleaned_indexed[pbr_col].interpolate(option='spline')

        sum_hour_cleaned_indexed = sum_hour_cleaned_indexed.reset_index(level=0)
        new_col = list(sum_hour_cleaned.columns)
        sum_hour_cleaned_indexed=sum_hour_cleaned_indexed[new_col]
        return sum_hour_cleaned_indexed
    else:
        print('No columns with missing values')
        return sum_hour_cleaned

cum_Dayly_cleaned = missingValue_imputation(tmp_CumDay_df)

No columns with missing values


In [38]:
cum_Dayly_cleaned.head()

message,time_stamp,date_time,unit,trial_no,species,D_CD_gpl,Co2_Conv,D_CO2_g,D_CO2_gpm,D_Co2_Conv_gpg,...,D_Harvest_l,D_Harvest_Algae_g,D_PAR_molpm,D_eff_percent,D_Productivity_g,D_Productivity_gpm,PAR_2Algae_Conv,R_Surface_m,R_Biomass_kg,R_Volume_l
0,1625281200,2021-07-03 05:00:00,6,3,11,0.30,0.6945,5.37,27.0,2.8,...,0.000,0.000,18.9,2.3,1.89,9.500000,0.07961,0.2,0.0018,6.00
1,1625367600,2021-07-04 05:00:00,6,3,11,0.82,0.6945,10.42,52.0,2.8,...,0.885,0.345,34.0,2.4,3.68,18.400000,0.07961,0.2,0.0052,6.27
2,1625454000,2021-07-05 05:00:00,6,3,11,0.94,0.6945,9.12,46.0,2.9,...,2.468,2.358,34.0,2.1,3.18,15.900000,0.07961,0.2,0.0060,6.37
3,1625540400,2021-07-06 05:00:00,6,3,11,0.99,0.6945,9.24,46.0,2.8,...,2.874,2.940,34.0,2.2,3.25,16.200001,0.07961,0.2,0.0063,6.39
4,1625626800,2021-07-07 05:00:00,6,3,11,1.00,0.6945,9.24,46.0,2.8,...,3.090,3.216,34.0,2.2,3.25,16.299999,0.07961,0.2,0.0063,6.30


In [39]:
# exporting the preprocessed data into db pbr
def pbr_exporttoDB(sumHour_preprocessed):
    connection = dBCon_Maria()
    cursor=connection.cursor()      
    cols = "`,`".join([str(i) for i in sumHour_preprocessed.columns.tolist()])
    for i,row in sumHour_preprocessed.iterrows():
        sql = "INSERT INTO `tbl_cumulative_per_day` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cursor.execute(sql, tuple(row))

    connection.commit()
    connection.close()


pbr_exporttoDB(cum_Dayly_cleaned)